# Testing Implementations
Although I plan to use real-world data to test the naive bayes classifier algorithm I'm building, 
it will likely be far quicker to use toy datasets to experiment with the Multinomial and Gaussian flavors of 
naive bayes, which is the purpose of this notebook

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

### Multinomial Naive Bayes
For the multinomial version of naive bayes, I need a dataset that contains discrete features, such as counts. I'll be using a dataset of IMDB reviews labelled either positive or negative. The labelled data is a text file with a 1 or 0 at the end of the line denoting a positive or negative review, respectively.

In [2]:
# I have removed punctuation and excess whitespace to
# prevent certain words from being differentiated, such
# as "very," and "very"

imdb_cols = ["review", "sentiment"]
imdb = pd.read_csv("imdb_labelled.txt", sep="\t", names=imdb_cols)
print(imdb["review"][344])

imdb["review"] = imdb["review"].str.strip()
imdb["review"] = imdb["review"].str.replace(r"[^\w\s-]", "")
imdb["review"] = imdb["review"].str.replace(r"\-", " ")
imdb["review"] = imdb["review"].str.replace(r"\s{2,}", " ")
imdb["review"] = imdb["review"].str.lower()

imdb.head()

While you don't yet hear Mickey speak, there are tons of sound effects and music throughout the film--something we take for granted now but which was a huge crowd pleaser in 1928.  


,review,sentiment
0,a very very very slow moving aimless movie abo...,0
1,not sure who was more lost the flat characters...,0
2,attempting artiness with black white and cleve...,0
3,very little music or anything to speak of,0
4,the best scene in the movie was when gerardo i...,1


In [3]:
print(imdb["review"][344])

while you dont yet hear mickey speak there are tons of sound effects and music throughout the film something we take for granted now but which was a huge crowd pleaser in 1928


In [4]:
# use sklearn's count vectorizer to create vectors for each review
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(imdb["review"])
y = imdb["sentiment"]

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(598, 3114)
(150, 3114)
(598,)
(150,)


The probability of being a good review is $\frac{312}{598}$, while the probability of being a bad review is $\frac{286}{598}$

In other words:

$P(Good) \approx 0.522$

$P(Bad) \approx 0.478$

In [5]:
# determining the prior probabilities for good and bad reviews
print(y_train.value_counts())
y_train.value_counts(normalize=True)

1    312
0    286
Name: sentiment, dtype: int64


1    0.521739
0    0.478261
Name: sentiment, dtype: float64

Now I want to examine the probability of a particular word being in a bad review. In this case, I'll be looking at the word "bad"

In [6]:
# create a matrix with "feature names"
words = vectorizer.get_feature_names()
term_matrix = pd.DataFrame(X_train.toarray(), columns=words)
term_matrix.head()

,010,10,1010,110,12,13,15,17,18th,1928,...,your,youre,yourself,youthful,youtube,youve,yun,zillion,zombie,zombiez
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# add the movie review label for reference
term_matrix["review_of_movie"] = y_train.values
term_matrix.head()

,010,10,1010,110,12,13,15,17,18th,1928,...,youre,yourself,youthful,youtube,youve,yun,zillion,zombie,zombiez,review_of_movie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
# get all bad and good reviews
bad = term_matrix[term_matrix["review_of_movie"] == 0]
good = term_matrix[term_matrix["review_of_movie"] == 1]

# total number of words in each
print(f"Total words in bad: {bad.sum().sum()}")
print(f"Total words in good: {good.sum().sum()}")

Total words in bad: 6003
Total words in good: 5954


How many times does the word bad occur in good and bad reviews?

In [70]:
print(f"Total times 'bad' appears in bad reviews: {bad.bad.sum()}")
print(f"Total times 'bad' appears in good reviews: {good.bad.sum()}")

Total times 'bad' appears in bad reviews: 50
Total times 'bad' appears in good reviews: 7


The probability that we will observe the word "bad" given that it was seen in a bad review is $\frac{50}{6003}$, while the probability that you might observe it in a good review is $\frac{7}{5954}$. That is:

$P(bad|Bad) \approx 0.008$

$P(bad|Good) \approx 0.001$

These numbers will need to be acquired for each word and label using the numpy array

In [153]:
X_train[(y_train.values == 1)].toarray().sum() + X_train[(y_train.values == 0)].toarray().sum()

11645

In [162]:
term_matrix.sum().sum() - 312

11645

In [177]:
# testing numpy's boolean indexing to see if
# it works the way I think it does
test_x = np.array([[0, 1, 1],
                   [1, 2, 1],
                   [6, 3, 1],
                   [1, 4, 1]])
test_y = np.array([0, 1, 1, 0])

# testing numpy's sum functions
test_x[(test_y == 1)].sum(axis=0)[1]

5

Now I need to know how calculating probabilities is going to work without using pandas explicitly. _Note_: The `y_train` values are a pandas series

In [203]:
# number of times "bad" appears in bad reviews
bad_in_bad = X_train[y_train.values == 0].toarray().sum(axis=0)[238]

# number of times "bad" appears in good reviews
bad_in_good = X_train[y_train.values == 1].toarray().sum(axis=0)[238]


total_words_in_good = X_train[y_train.values == 1].sum()
total_words_in_bad = X_train[y_train.values == 0].sum()

print("P(bad|Bad): %.3f" % (bad_in_bad / total_words_in_bad))
print("P(good|Bad): %.3f" % (bad_in_good / total_words_in_good))

P(bad|Bad): 0.008
P(good|Bad): 0.001


The calculation above is the same as what I arrived at previously, using pandas. Now I need to figure out how to get and store these probabilities for each word, for each class

In [ ]:
# first I need to be able to count the classes in the dependent variable
classes, counts = np.unique(y_train.values, return_counts=True)
dict(zip(classes, counts))

# I need to be able to store the individual probabilites for 
# each word, given a class
class_probabilities = {c:{} for c in classes}
class_probabilities

for c in class_probabilities:
    total_words = X_train[y_train.values == c].sum()
    for w in range(X_train.shape[1]):
        word_occurrences = X_train[y_train.values == c].toarray().sum(axis=0)[w]
    
        class_probabilities[c][w] = (word_occurrences / total_words)
    
class_probabilities        

The word "bad" was the 238th word of the transposed term matrix. This should align with the new `class_probabilites` dictionary, which contains the probabilites for each word, given each class:

In [248]:
print("P(bad|Bad): %.3f" % class_probabilities[0][238])
print("P(bad|Bad): %.3f" % class_probabilities[1][238])

P(bad|Bad): 0.008
P(bad|Bad): 0.001


Now that the probabilities are reliably stored and indexed we should be able to use use Bayes' Theorem with naive assumptions (that is, assuming that each word is independent of all others. We're assuming no word affects the amount or appe

### Gaussian Naive Bayes
For the Gaussian Naive Bayes, i'll be using the famed Iris dataset to provide continuous data for classification. 

In [5]:
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data")

In [6]:
iris.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
